# Déploiement des campagnes et des filtres<a class="anchor" id="top"></a>

Dans ce bloc-notes, vous allez déployer et interagir avec les campagnes dans Amazon Personalize.

1. [Introduction](#intro)
1. [Créer des campagnes](#create)
1. [Interagir avec les campagnes](#interact)
1. [Recommandations par lots](#batch)
1. [Récapitulatif](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Retour au début](#top)

À ce stade, vous devriez avoir plusieurs solutions et au moins une version de solution pour chacune d'elles. Une fois qu'une version de la solution est créée, il est possible d'obtenir des recommandations de cette dernière et de s'imprégner de son comportement général.

Ce bloc-notes déploie tout d'abord chacune des versions de la solution du bloc-notes précédent dans des campagnes individuelles. Une fois qu'elles sont actives, des ressources sont prévues pour interroger les recommandations, ainsi que des fonctions d'aide pour transformer le résultat en données plus lisibles par l'homme. 

Comme vous avec votre client sur Amazon Personalize, vous pouvez modifier les fonctions d'aide pour les adapter à la structure de leurs fichiers d'entrée de données afin que le rendu supplémentaire fonctionne.

Pour commencer, nous devons une fois de plus importer des bibliothèques, charger les valeurs des bloc-notes précédents et charger le kit SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Créer des campagnes <a class="anchor" id="create"></a>
[Retour au début](#top)

Une campagne est une version de la solution hébergée, un point de terminaison que vous pouvez interroger pour obtenir des recommandations. La tarification est fixée en estimant la capacité de débit (demandes de personnalisation de l'utilisateur par seconde). Lors du déploiement d'une campagne, vous définissez une valeur minimale de débit par seconde (TPS). Ce service, comme beaucoup d'autres au sein d'AWS, évoluera automatiquement en fonction de la demande. Néanmoins, si la latence est stratégique, vous voudrez peut-être prendre des dispositions pour une demande plus importante. Pour cette POC et cette démo, tous les seuils de débit minimum sont définis à 1. Pour plus d'informations, reportez-vous à la [page de tarification](https://aws.amazon.com/personalize/pricing/).

Commençons à déployer les campagnes.

### Personnalisation de l'utilisateur

Déployez une campagne pour votre version de la solution de personnalisation de l'utilisateur. Le déploiement d'une campagne peut prendre environ 10 minutes. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Cependant, la tâche bloquerait l'exécution d'autres cellules, et le but ici est de créer de multiples campagnes. Nous allons donc mettre en place la boucle while pour toutes les campagnes plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [4]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-userpersonalization",
    solutionVersionArn = userpersonalization_solution_version_arn,
    minProvisionedTPS = 1
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print(json.dumps(userpersonalization_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "ea0b63f5-9efa-4abd-bc88-dd5212e85668",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:29 GMT",
      "x-amzn-requestid": "ea0b63f5-9efa-4abd-bc88-dd5212e85668",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS

Déployez une campagne pour la version de votre solution SIMS. Le déploiement d'une campagne peut prendre environ 10 minutes. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Cependant, la tâche bloquerait l'exécution d'autres cellules, et le but ici est de créer de multiples campagnes. Nous allons donc mettre en place la boucle while pour toutes les campagnes plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [5]:
sims_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
print(json.dumps(sims_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-SIMS",
  "ResponseMetadata": {
    "RequestId": "8d8f06e3-2699-4bcd-bb60-9389aff0fcf4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:29 GMT",
      "x-amzn-requestid": "8d8f06e3-2699-4bcd-bb60-9389aff0fcf4",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Classement personnalisé

Déployez une campagne pour la version de votre solution de classement personnalisée. Le déploiement d'une campagne peut prendre environ 10 minutes. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Cependant, la tâche bloquerait l'exécution d'autres cellules, et le but ici est de créer de multiples campagnes. Nous allons donc mettre en place la boucle while pour toutes les campagnes plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [6]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "c28bff59-c214-4f57-85d4-5e2643a03413",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:29 GMT",
      "x-amzn-requestid": "c28bff59-c214-4f57-85d4-5e2643a03413",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Afficher le statut de création de la campagne

Voici, comme promis, comment afficher les mises à jour du statut dans la console :

* La console AWS devrait déjà être ouverte dans un autre onglet du navigateur, après l'ouverture de cette instance du bloc-notes. 
* Passez à cet onglet et recherchez en haut le service `Personalize`, puis référez-vous à cette page de service. 
* Cliquez sur `View dataset groups`.
* Cliquez sur le nom de votre groupe de jeux de données, très probablement un nom contenant POC.
* Cliquez sur `Campaigns`.
* Vous verrez maintenant une liste de toutes les campagnes que vous avez créées ci-dessus, y compris une colonne avec le statut de la campagne. Une fois qu'elle est `Active`, votre campagne est prête à être interrogée.

Ou exécutez simplement la cellule ci-dessous pour surveiller le statut de création de la campagne.

In [7]:
in_progress_campaigns = [
    userpersonalization_campaign_arn,
    sims_campaign_arn,
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-SIMS
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-userpersonalization
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-rerank


## Créer des filtres <a class="anchor" id="interact"></a>
[Retour au début](#top)

Toutes les campagnes étant déployées et actives, nous pouvons à présent créer des filtres. Les filtres peuvent être créés à la fois pour les articles et les événements. Parmi les cas d'utilisation courants des filtres dans la vidéo à la demande, figurent les suivants :

Filtres catégoriels basés sur les métadonnées de l'article – Les métadonnées de l'article contiennent souvent des informations sur le titre, telles que le genre, le mot clé, l'année, la décennie, etc. En filtrant ces données, vous pouvez obtenir des recommandations, par exemple des films d'action.

Événements – vous pouvez vouloir filtrer certains événements et fournir des résultats basés sur ces événements, par exemple en déplaçant un titre d'une recommandation "suggestions à regarder" à une recommandation "regarder à nouveau".

Examinons les métadonnées des articles et les interactions des utilisateurs, afin de déterminer le type de filtres que nous pouvons créer.

In [8]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)
#interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
items_df.head(10)
#interactions_df.head(10)

,GENRE,YEAR
ITEM_ID,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995
2,Adventure|Children|Fantasy,1995
3,Comedy|Romance,1995
4,Comedy|Drama|Romance,1995
5,Comedy,1995
6,Action|Crime|Thriller,1995
7,Comedy|Romance,1995
8,Adventure|Children,1995
9,Action,1995


Nous voulons à présent déterminer les genres à filtrer. Pour cela, nous avons besoin d'une liste de tous les genres. Tout d'abord, nous collecterons toutes les valeurs uniques de la colonne GENRE, puis répartirons les chaînes sur `|` si elles existent, chacune sera alors ajoutée à une longue liste qui sera convertie en un ensemble pour plus d'efficacité. Ce jeu de données sera ensuite transformé en une liste afin qu'il puisse être itéré et, nous pourrons alors utiliser l'API de création de filtre.

In [9]:
unique_genre_field_values = items_df['GENRE'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [10]:
genres_to_filter

['Animation',
 'Thriller',
 'Sci-Fi',
 'IMAX',
 'Horror',
 'Comedy',
 'Adventure',
 'Musical',
 'Drama',
 'Romance',
 'Fantasy',
 'War',
 'Documentary',
 'Crime',
 'Western',
 'Film-Noir',
 'Mystery',
 'Action',
 'Children']

Nous disposons ainsi de tous les genres qui existent dans notre jeu de données. La limite autorisée de Personalize est actuellement de 10 filtres au total. Comme nous avons un plus grand nombre de genres, nous en sélectionnerons 7 au hasard pour laisser de la place pour 2 filtres basés sur l'interaction plus tard et un filtre supplémentaire pour les recommandations basées sur l'année.

In [11]:
genres_to_filter = random.sample(genres_to_filter, 7)
genres_to_filter

['Film-Noir', 'Fantasy', 'Western', 'Documentary', 'Comedy', 'Action', 'IMAX']

Créez ensuite une liste pour les filtres de genre de métadonnées, puis créez les filtres réels avec les cellules ci-dessous. Notez que cette opération prendra quelques minutes.

In [12]:
# Create a list for the filters:
meta_filter_arns = []

In [13]:
# Iterate through Genres
for genre in genres_to_filter:
    # Start by creating a filter
    try:
        createfilter_response = personalize.create_filter(
            name=genre,
            datasetGroupArn=dataset_group_arn,
            filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
        )
        # Add the ARN to the list
        meta_filter_arns.append(createfilter_response['filterArn'])
        print("Creating: " + createfilter_response['filterArn'])
    
    # If this fails, wait a bit
    except ClientError as error:
        # Here we only care about raising if it isnt the throttling issue
        if error.response['Error']['Code'] != 'LimitExceededException':
            print(error)
        else:    
            time.sleep(120)
            createfilter_response = personalize.create_filter(
                name=genre,
                datasetGroupArn=dataset_group_arn,
                filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
            )
            # Add the ARN to the list
            meta_filter_arns.append(createfilter_response['filterArn'])
            print("Creating: " + createfilter_response['filterArn'])

Creating: arn:aws:personalize:us-east-1:136455442858:filter/Film-Noir
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Fantasy
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Western
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Documentary
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Comedy
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Action
Creating: arn:aws:personalize:us-east-1:136455442858:filter/IMAX


Créons également deux filtres d'événements pour le contenu regardé et non regardé.

In [14]:
# Create a dataframe for the interactions by reading in the correct source CSV
interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
interactions_df.head(10)

,ITEM_ID,TIMESTAMP,EVENT_TYPE
USER_ID,,,
429,222,828124615,watch
429,227,828124615,click
429,595,828124615,watch
429,592,828124615,watch
429,590,828124615,watch
429,434,828124615,watch
429,421,828124615,watch
429,225,828124615,click
429,343,828124615,click


Créons également deux filtres d'événements pour le contenu regardé et non regardé.

In [15]:
createwatchedfilter_response = personalize.create_filter(name='watched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )

createunwatchedfilter_response = personalize.create_filter(name='unwatched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='EXCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )


Enfin, puisque nous disposons désormais de l'année dans les métadonnées de nos articles, créons un filtre de décennie pour recommander uniquement les films sortis au cours d'une décennie donnée. Pour cet atelier, nous choisirons le cinéma des années 1970. 

In [16]:
createdecadefilter_response = personalize.create_filter(name='1970s',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Items.YEAR >= 1970 AND Items.YEAR < 1980'
    )

Avant de terminer, nous voudrons également ajouter ces filtres à une liste afin de pouvoir les utiliser ultérieurement.

In [17]:
interaction_filter_arns = [createwatchedfilter_response['filterArn'], createunwatchedfilter_response['filterArn']]

In [18]:
decade_filter_arns = [createdecadefilter_response['filterArn']]

In [19]:
%store sims_campaign_arn
%store userpersonalization_campaign_arn
%store rerank_campaign_arn
%store meta_filter_arns
%store interaction_filter_arns
%store decade_filter_arns


Stored 'sims_campaign_arn' (str)
Stored 'userpersonalization_campaign_arn' (str)
Stored 'rerank_campaign_arn' (str)
Stored 'meta_filter_arns' (list)
Stored 'interaction_filter_arns' (list)
Stored 'decade_filter_arns' (list)


Vous êtes prêts à passer au dernier bloc-notes exploratoire : `05_Interacting_with_Campaigns_and_Filters.ipynb`. Ouvrez-le à partir du navigateur afin de pouvoir interagir avec les campagnes et obtenir des recommandations !